In [1]:
import time

### Load Data

In [2]:
# Parse the source
with open('dataset/THUOCL_chengyu.txt', 'r', encoding='utf-8') as source:
    idioms = [i for i in map(lambda x: x.split()[0], source.readlines()) if len(i) <= 4]

nodes = set([i[0] for i in idioms] + [i[-1] for i in idioms])
edges = [{"src": i[0], "dst": i[-1], "idiom": i} for i in idioms]

#### Create Graph

In [3]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", password="12345678")

cql = "CREATE \n" + ",".join("(%s : Node {char: \"%s\"})\n"%(n, n) for n in nodes)
for e in edges:
    cql += ",(%s)-[:idiom {idiom: '%s'}]->(%s)\n"%(e['src'], e["idiom"], e['dst'])

start = time.time()
graph.run("match (n) detach delete n")
graph.run(cql)
print("created graph, time elapsed: %.2fs"%(time.time() - start))

created graph, time elapsed: 9.38s


___
### Query Graph
##### Find next word

In [4]:
next_word = lambda word: graph.run("match ()-[:idiom {idiom: '%s'}]->()-[e:idiom]->() return e.idiom"%word)

df_next = next_word("坚定不移").to_data_frame()

print("found %i idioms"%len(df_next))
df_next.head()

found 3 idioms


,e.idiom
0,移花接木
1,移风易俗
2,移山填海


##### Find path

In [5]:
def find_path(idiom, length, limit):
    cql = "match ()-[e:idiom {idiom: '%s'}]->()"%idiom
    path = "".join(["-[e%i:idiom]->()"%i for i in range(length)])
    ret = ", ".join(["e%i.idiom"%i for i in range(length)])
    return graph.run(cql + path + " return e.idiom, " + ret + " limit %i"%limit).to_data_frame()

In [6]:
start = time.time()
df_paths = find_path(idiom="坚定不移", length=5, limit=100000)

print("found %i paths, time elapsed: %.2fs"%(len(df_paths), time.time() - start))
df_paths.head(10)

found 6833 paths, time elapsed: 0.23s


,e.idiom,e0.idiom,e1.idiom,e2.idiom,e3.idiom,e4.idiom
0,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后会有期
1,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人
2,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后生小子
3,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后生可畏
4,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后起之秀
5,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后悔不及
6,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后会无期
7,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后发制人
8,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后顾之忧
9,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后来居上


In [7]:
start = time.time()
df_paths = find_path(idiom="坚定不移", length=10, limit=100000)

print("found %i paths, time elapsed: %.2fs"%(len(df_paths), time.time() - start))
df_paths.to_csv("output/坚定不移.csv", index=False)
df_paths.head(10)

found 100000 paths, time elapsed: 8.98s


,e.idiom,e0.idiom,e1.idiom,e2.idiom,e3.idiom,e4.idiom,e5.idiom,e6.idiom,e7.idiom,e8.idiom,e9.idiom
0,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气吞山河,河东狮吼
1,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气吞山河,河清海晏
2,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气壮如牛,牛鬼蛇神
3,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气壮如牛,牛刀小试
4,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气壮如牛,牛郎织女
5,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气壮如牛,牛之一毛
6,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气壮如牛,牛头马面
7,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气壮如牛,牛高马大
8,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气数已尽,尽善尽美
9,坚定不移,移花接木,木牛流马,马首是瞻,瞻前顾后,后继有人,人神共愤,愤愤不平,平心静气,气数已尽,尽释前嫌
